In [1]:
#import the module
import json
import pandas as pd
import requests

#retrieve theMovieDB.org API key
def get_keys(path):
    '''
    Init signature: get_keys(filename=None)
    Docstring:     
    get_keys(filename='') -> dict
    Retrieve API key stored on file.  return a dictionary.
    
    Parameters
    ----------
    filename: Loation of the json file where the API key is stored.
      
    '''
    with open(path) as f:
        return json.load(f)


In [38]:
# import the api key for movieDB using the get_keys function
keys = get_keys('moviedb.json')

#assign API value to variabloe
api_key = keys['api_key']


To be deleted
https://api.themoviedb.org/3/discover/movie?api_key=da53dbf2111373cbf50f44192834054a&language=en-US
&sort_by=popularity.desc&include_adult=false&include_video=false&page=3&primary_release_date.gte=2010-01-01

In [34]:
# Define get request parameters
url = 'https://api.themoviedb.org/3/discover/movie?'
langauge = 'en-US'
sort_by = 'desc'
include_adult = 'false'
include_video = 'false'
primary_release_date_gt = '2010-01-01'

url_param = {
                'api_key': api_key,
                'language': langauge,
                'sort_by': sort_by,
                'include_adult': include_adult,
                'include_video': include_video,
                'primary_release_date.gt': primary_release_date_gt
            }

# url_param.update({'page': 1})
# test = requests.get(url, params=url_param)
# test.json()['results']

# API only return 1 page with 20 results.  Set the loop to 500 times since the max number of pages allowed is 10,000.
for i in range(1,500): 
    # update the page number using the iterate value
    url_param.update({'page': i})
    # create the response ojb.
    resp = requests.get(url, params=url_param)
    # if request status is sucessful and the value of key ['page'] is 1
    if resp.status_code == 200 and resp.json()['page'] == 1:
        # create the DataFrame
        df = pd.DataFrame.from_dict(resp.json()['results'])
    # if request status is sucessful and able to read the value of key ['page']
    elif resp.status_code == 200 and resp.json().get('page', 10000) < 10000:
        print(resp.json()['page'])
        a = pd.DataFrame.from_dict(resp.json()['results'])
        df = pd.concat([df, a], sort=False)
    # if request status is sucessful and unable to read the value of key ['page'], assign value 10,000 and quit the loop  
    elif resp.status_code == 200 and resp.json().get('page', 10000) == 10000:
        break


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [35]:
df.shape

(9980, 14)

In [51]:
# reset dataframe index due to duplicates from concat of dataframe
df2 = df.reset_index()

# remove column index as well as irrelevent columns
df2.drop(['index', 'video', 'poster_path', 'backdrop_path'], axis=1, inplace=True)


In [63]:
# create a function to send request to pull movie details
# define get request parameters

def get_movie_rev(movie_id):
    '''
    Init signature: get_keys(movie_id)
    Docstring:     
    get_movie_rev -> float
    Retrieve 
    
    Parameters
    ----------
    url: base TMDB API base URL
    api_key: TMDB api_key
    movie_id: TMDB movie id

    '''
    #https://api.themoviedb.org/3/movie/51220?api_key=da53dbf2111373cbf50f44192834054a&language=en-US
    url = 'https://api.themoviedb.org//3/movie/' + str(movie_id) + '?'
    url_param = {
                    'api_key': api_key,
                    'language': 'en-US',
                }
    respond = requests.get(url, params=url_param)
    return respond

res_rev = get_movie_rev(51220)
# res_rev.status_code
res_rev.headers

{'Content-Type': 'text/html', 'Content-Length': '166', 'Connection': 'keep-alive', 'Date': 'Tue, 03 Mar 2020 13:14:58 GMT', 'Server': 'openresty', 'X-Cache': 'Error from cloudfront', 'Via': '1.1 b96e96fef61e11f1c624b4bdd0ad939e.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'JFK51-C1', 'X-Amz-Cf-Id': 'NlfxnkK4XAaYEVnweTXOztZ3S2-N8GAjSGJAhRwLlF6c12lgEaA7cA==', 'Age': '263'}

In [ ]:
res_rev.json()

In [13]:
r = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=da53dbf2111373cbf50f44192834054a&language=en-US&sort_by=popularity.desc&include_adult=false&include_video=false&page=1&primary_release_date.gte=2010-01-01')
r2 = requests.get('https://api.themoviedb.org/3/discover/movie?api_key=da53dbf2111373cbf50f44192834054a&language=en-US&sort_by=popularity.desc&include_adult=false&include_video=false&page=2&primary_release_date.gte=2010-01-01')

In [ ]:
# r3 = pd.concat([pd.DataFrame.from_dict(r.json()['results']), pd.DataFrame.from_dict(r2.json()['results'])])
r3.reset_index()

In [39]:
type(r)

requests.models.Response

In [ ]:


for i, j in enumerate(json_list):
    if i == 0:
        print(j['results'])
        df = pd.DataFrame(j['results'])        
    else:
        df.append(pd.DataFrame(j['results']))




In [ ]:
Init signature: pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
Docstring:     
Two-dimensional size-mutable, potentially heterogeneous tabular data
structure with labeled axes (rows and columns). Arithmetic operations
align on both row and column labels. Can be thought of as a dict-like
container for Series objects. The primary pandas data structure.

Parameters
----------
data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
    Dict can contain Series, arrays, constants, or list-like objects

    .. versionchanged :: 0.23.0
       If data is a dict, column order follows insertion-order for
       Python 3.6 and later.

    .. versionchanged :: 0.25.0
       If data is a list of dicts, column order follows insertion-order
       Python 3.6 and later.

index : Index or array-like
    Index to use for resulting frame. Will default to RangeIndex if
    no indexing information part of input data and no index provided
columns : Index or array-like
    Column labels to use for resulting frame. Will default to
    RangeIndex (0, 1, 2, ..., n) if no column labels are provided
dtype : dtype, default None
    Data type to force. Only a single dtype is allowed. If None, infer
copy : boolean, default False
    Copy data from inputs. Only affects DataFrame / 2d ndarray input

See Also
--------
DataFrame.from_records : Constructor from tuples, also record arrays.
DataFrame.from_dict : From dicts of Series, arrays, or dicts.
DataFrame.from_items : From sequence of (key, value) pairs
    read_csv, pandas.read_table, pandas.read_clipboard.

Examples
--------
Constructing DataFrame from a dictionary.

>>> d = {'col1': [1, 2], 'col2': [3, 4]}
>>> df = pd.DataFrame(data=d)
>>> df
   col1  col2
0     1     3
1     2     4

Notice that the inferred dtype is int64.

>>> df.dtypes
col1    int64
col2    int64
dtype: object

To enforce a single dtype:

>>> df = pd.DataFrame(data=d, dtype=np.int8)
>>> df.dtypes
col1    int8
col2    int8
dtype: object

Constructing DataFrame from numpy ndarray:

>>> df2 = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
...                    columns=['a', 'b', 'c'])
>>> df2
   a  b  c
0  1  2  3
1  4  5  6
2  7  8  9
File:           ~/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py
Type:           type
Subclasses:     SparseDataFrame, SubclassedDataFrame